# Import Packages

In [1]:
from warnings import simplefilter
simplefilter('ignore')

from performStockAnalysis.main import select_kode_to_model, develop_models_for_selected_kode

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

NameError: name 'pd' is not defined

In [ ]:
selected_kode = select_kode_to_model(4)

In [ ]:
all_model_performances_10_days, all_model_performances_15_days = develop_models_for_selected_kode(selected_kode)

In [ ]:
all_model_performances_10_days

In [ ]:
all_model_performances_15_days